In [18]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, KFold
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from lightgbm import LGBMRegressor
import lightgbm as lgb
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV, ShapleyImportanceEvaluator
from optuna.integration.lightgbm import LightGBMTunerCV #lgbm 전용 tunner 가 존재
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import ExtraTreeClassifier

# 데이터 불러오기

In [34]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')
y_train = train['전화해지여부']
X_train = train.drop(['전화해지여부', 'ID'], axis = 1)
X_test = test.drop('ID', axis = 1)

# 전처리

- 이상치로 표현할만한 값이 없을것이라 판단함

In [35]:
X_train.isnull().sum()

가입일        0
음성사서함이용    0
주간통화시간     0
주간통화횟수     0
주간통화요금     0
저녁통화시간     0
저녁통화횟수     0
저녁통화요금     0
밤통화시간      0
밤통화횟수      0
밤통화요금      0
상담전화건수     0
dtype: int64

- 결측지는 존재하지 않음

In [36]:
a = X_train.columns
for i in a:
    print(X_train[i].value_counts())

223     1307
222     1048
23       757
93       648
203      641
        ... 
15         1
494        1
217        1
215        1
2113       1
Name: 가입일, Length: 345, dtype: int64
0       10988
1       10852
23        825
22        669
29        610
        ...  
117         1
13          1
116         1
111         1
1112        1
Name: 음성사서함이용, Length: 76, dtype: int64
224.0    204
223.8    202
223.5    198
223.7    195
224.1    194
        ... 
198.2      1
423.0      1
131.7      1
476.7      1
164.7      1
Name: 주간통화시간, Length: 2771, dtype: int64
223    1421
203    1310
93     1218
222    1176
202    1067
       ... 
469       1
157       1
263       1
465       1
486       1
Name: 주간통화횟수, Length: 239, dtype: int64
23.5    322
23.7    318
23.6    307
23.4    305
23.8    297
       ... 
2.4       1
15.9      1
15.2      1
1.0       1
53.6      1
Name: 주간통화요금, Length: 735, dtype: int64
223.7    194
223.9    194
223.3    190
223.5    187
223.1    180
        ... 
459.6      1
130.5  

# 피처만들기

In [37]:
X_train['가입월'] = X_train['가입일']//30
X_test['가입월'] = X_test['가입일']//30

X_train['가입년'] = X_train['가입일']//365
X_test['가입년'] = X_test['가입일']//365


X_train['총통화시간'] = X_train['주간통화시간'] + X_train['저녁통화시간'] + X_train['밤통화시간']
X_test['총통화시간'] = X_test['주간통화시간'] + X_test['저녁통화시간'] + X_test['밤통화시간']

X_train['총통화요금'] = X_train['주간통화요금'] + X_train['저녁통화요금'] + X_train['밤통화요금']
X_test['총통화요금'] = X_test['주간통화요금'] + X_test['저녁통화요금'] + X_test['밤통화요금']

X_train['총통화횟수'] = X_train['주간통화횟수'] + X_train['저녁통화횟수'] + X_train['밤통화횟수']
X_test['총통화횟수'] = X_test['주간통화횟수'] + X_test['저녁통화횟수'] + X_test['밤통화횟수']


X_train['통화시간곱'] = X_train['주간통화시간'] * X_train['저녁통화시간'] * X_train['밤통화시간']
X_test['통화시간곱'] = X_test['주간통화시간'] * X_test['저녁통화시간'] * X_test['밤통화시간']
X_train['총통화횟수'] = X_train['주간통화횟수'] + X_train['저녁통화횟수'] + X_train['밤통화횟수']
X_test['총통화횟수'] = X_test['주간통화횟수'] + X_test['저녁통화횟수'] + X_test['밤통화횟수']
X_train['통화요금곱'] = X_train['주간통화요금'] * X_train['저녁통화요금'] * X_train['밤통화요금']
X_test['통화요금곱'] = X_test['주간통화요금'] * X_test['저녁통화요금'] * X_test['밤통화요금']


In [38]:
X_train['통화횟수곱'] = X_train['주간통화횟수'] * X_train['저녁통화횟수'] * X_train['밤통화횟수']
X_test['통화횟수곱'] = X_test['주간통화횟수'] * X_test['저녁통화횟수'] * X_test['밤통화횟수']

a = []
for i in X_train['가입일']:
    if i<100:
        a.append(0)
    else:
        a.append(1)
X_train['장기'] = a

b = []
for i in X_test['가입일']:
    if i<100:
        b.append(0)
    else:
        b.append(1)
X_test['장기'] = b

X_train['상담+음성'] = X_train['상담전화건수']*10 + X_train['음성사서함이용']
X_test['상담+음성'] = X_test['상담전화건수']*10 + X_test['음성사서함이용']

In [45]:
X_train.to_csv('X_train_2.csv',encoding='cp949', index=False)
X_test.to_csv('X_test_2.csv',encoding='cp949', index=False)